<a href="https://colab.research.google.com/github/iwatake2222/colaboratory_study/blob/master/utility/tfrecord2csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convert TFRecord format to csv (Google cloud Auto ML Vision) format
## Preparation

In [2]:
from google.colab import drive 
drive.mount('/content/drive')

%tensorflow_version 2.x
import tensorflow as tf
print(tf.__version__)

2.1.0-rc1


## Functions

In [0]:
from PIL import Image
import os
import tensorflow as tf
from io import BytesIO
import csv
import argparse

URI_GOOGLE_STORAGE = "gs://golden-memory-123456-vcm/test/"
OUTPUT_DIR_NAME = "genarated_csv_dataset"

os.makedirs(OUTPUT_DIR_NAME, exist_ok=True)

def get_features_from_record(record_name):
	ret = []
  # for raw_record in tf.python_io.tf_record_iterator(record_name):
	for raw_record in tf.data.TFRecordDataset(record_name).take(1):
		example = tf.train.Example()
#		example.ParseFromString(raw_record)
		example.ParseFromString(raw_record.numpy())
		print(example)
		filename = example.features.feature["image/filename"].bytes_list.value[0]
#		source_id = example.features.feature["image/source_id"].bytes_list.value[0]
#		format_ = example.features.feature["image/format"].bytes_list.value[0]
#		height = example.features.feature["image/height"].int64_list.value[0]
#		width = example.features.feature["image/width"].int64_list.value[0]	
		xmax_list = example.features.feature["image/object/bbox/xmax"].float_list.value
		xmin_list = example.features.feature["image/object/bbox/xmin"].float_list.value
		ymax_list = example.features.feature["image/object/bbox/ymax"].float_list.value
		ymin_list = example.features.feature["image/object/bbox/ymin"].float_list.value
#		class_label_list = example.features.feature["image/object/class/label"].int64_list.value
		class_text_list = example.features.feature["image/object/class/text"].bytes_list.value
#		difficult_list = example.features.feature["image/object/difficult"].int64_list.value
#		truncated_list = example.features.feature["image/object/truncated"].int64_list.value
#		view_list = example.features.feature["image/object/view"].bytes_list.value
		image = example.features.feature["image/encoded"].bytes_list.value[0]

		img = Image.open(BytesIO(image))
		img.save(os.path.join(OUTPUT_DIR_NAME, filename.decode("utf-8")))
		if len(xmin_list) > 0:
			for (xmin, xmax, ymin, ymax, class_text) in zip(xmin_list, xmax_list, ymin_list, ymax_list, class_text_list):
				ret.append(["UNASSIGNED", URI_GOOGLE_STORAGE + filename.decode("utf-8"), class_text.decode("utf-8"), xmin, ymin,"","",xmax,ymax,"",""])
		else:
			ret.append(["UNASSIGNED", URI_GOOGLE_STORAGE + filename.decode("utf-8"), "", "", "","","","","","",""])
	return ret

def convert_from_tfrecord_to_csv(path_to_dir):
	with open(os.path.join(OUTPUT_DIR_NAME, "annotation.csv"), 'w', newline="\n") as f:
		writer = csv.writer(f)
		tfrecord_files = [file for file in os.listdir(path_to_dir) if ".tfrecord" in file]
		for tfrecord_file in tfrecord_files:
			ret = get_features_from_record(os.path.join(path_to_dir, tfrecord_file))
			writer.writerows(ret)

## Execution
`input_dir` is the path to the directory containing *.tfrecord files

In [37]:
!cp "/content/drive/My Drive/data/beer/pet_faces_val.record-00001-of-00002" test.tfrecord
input_dir = "./"
convert_from_tfrecord_to_csv(input_dir)
!ls genarated_csv_dataset
!more genarated_csv_dataset/annotation.csv


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



20190922_201645.jpg  annotation.csv
UNASSIGNED,gs://golden-memory-123456-vcm/test/20190922_201645.jpg,beer,0.1995192
3191547394,0.15811966359615326,,,0.3173076808452606,0.45512819290161133,,
UNASSIGNED,gs://golden-memory-123456-vcm/test/20190922_201645.jpg,beer,0.328125,
0.15598291158676147,,,0.42548078298568726,0.4615384638309479,,
UNASSIGNED,gs://golden-memory-123456-vcm/test/20190922_201645.jpg,beer,0.4579326
808452606,0.1260683834552765,,,0.551682710647583,0.45512819290161133,,
UNASSIGNED,gs://golden-memory-123456-vcm/test/20190922_201645.jpg,beer,0.5949519
276618958,0.17521367967128754,,,0.707932710647583,0.5085470080375671,,
UNASSIGNED,gs://golden-memory-123456-vcm/test/20190922_201645.jpg,beer,0.6598557
829856873,0.5534188151359558,,,0.7836538553237915,0.9017093777656555,,
UNASSIGNED,gs://golden-memory-123456-vcm/test/20190922_201645.jpg,beer,0.5036057
829856873,0.5747863054275513,,,0.6069711446762085,0.9038461446762085,,
UNASSIGNED,gs://golden-memory-123456-vcm/test/20190922_20